In [1]:
import glob
import os
import struct
from nltk.tokenize import sent_tokenize

from tensorflow.core.example import example_pb2

In [2]:
file0 = '/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/train_000.bin'

In [3]:
files = sorted(os.listdir('/home/francisco/GitHub/cnn-dailymail/finished_files/chunked/'))

load word vecs

Iterate through chunks of the 

In [4]:
def returnBytes(reader_obj):
    len_bytes = reader_obj.read(8)
    str_len = struct.unpack('q', len_bytes)[0]
    e_s = struct.unpack("%ds" % str_len, reader.read(str_len))
    es = e_s[0]
    c = example_pb2.Example.FromString(es)
    article  = str(c.features.feature['article'].bytes_list.value[0])
    abstract = str(c.features.feature['abstract'].bytes_list.value[0])
    ab = sent_tokenize(abstract)
    clean_article = sent_tokenize(article)
    clean_abstract = '. '.join([' '.join(s for s in x.split() if s.isalnum()) for x in ''.join(ab).replace("<s>","").split("</s>")]).strip()    
    return clean_abstract, clean_article

In [5]:
reader = open(file0, 'rb')

In [6]:
ab0, ar0 =  returnBytes(reader)
ab1, ar1 =  returnBytes(reader)

In [7]:
ab1

'harry potter star daniel radcliffe gets 20m fortune as he turns 18 monday. young actor says he has no plans to fritter his cash away. radcliffe earnings from first five potter films have been held in trust fund.'

In [8]:
import sys
import pandas as pd
sys.path.append('../src')
import data_io, params, SIF_embedding
import show_chunked

In [9]:
# input
wordfile = '/home/francisco/GitHub/SIF/data/glove.840B.300d.txt'   # word vector file, can be downloaded from GloVe website
weightfile = '/home/francisco/GitHub/SIF/auxiliary_data/enwiki_vocab_min200.txt' # each line is a word and its frequency
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 0
# rmpc = 1 # they usually set 1 ; number of principal components to remove in SIF weighting scheme

In [10]:
# load word vectors
(words, We) = data_io.getWordmap(wordfile)

In [11]:
# load word weights
word2weight = data_io.getWordWeight(weightfile, weightpara) # word2weight['str'] is the weight for the word 'str'
weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word

In [12]:
# set parameters
params = params.params()
params.rmpc = rmpc

In [13]:
sentences = ar0

In [14]:
# load sentences
x, m = data_io.sentences2idx(sentences, words) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
w = data_io.seq2weight(x, m, weight4ind) # get word weights

# get SIF embedding
embeddings = SIF_embedding.SIF_embedding(We, x, w, params) # embedding[i,:] is the embedding for sentence i

In [37]:
# Sentence embeddings
sdf = pd.DataFrame(sentences, columns=['sentence'])
sdf['summary'] = ab0
sdf.ix[1:, 'summary']  = ''
emb = pd.DataFrame(embeddings, columns=['emb_%i' % x for x in range(embeddings.shape[1])])
sdf = pd.concat([sdf, emb], axis=1)
sdf = sdf[[sdf.columns[1], sdf.columns[0]] + sdf.columns[2:].tolist()]